# tinygrad – minimalistyczna biblioteka głębokiego uczenia maszynowego

## Wprowadzenie

tinygrad to lekka, otwarta biblioteka stworzona przez George'a Hotza (tiny corp). Łączy **prostotę** mikrobiblioteki _micrograd_ Karpathy'ego z funkcjonalnością podobną do PyTorch'a. Ze względu na niewielki (10_000 linii), czytelny kod źródłowy jest polecana początkującym, którzy chcą zrozumieć wewnętrzne mechanizmy sieci neuronowych. Biblioteka jest w fazie alpha, ale zyskuje popularność (np. w projekcie OpenPilot jako backend GPU).

---

- micrograd to 'biblioteka', który implementuje zwykłą sieć neuronową w 100 linijkach kodu w Pythonie i to tyle, nie jest już utrzymywana
- Dla ludzi, którzy wiedzą jak trenować sieci neuronowe, ale chcą zrozumieć jak działają od strony implementacyjnej
- OpenPilot to open source software do autonomicznych samochodów, stworzony przez firme comma.ai, założoną przez George'a Hotza

---

## Historia i kontekst powstania

George Hotz stworzył tinygrad w 2020 roku jako alternatywę dla coraz bardziej skomplikowanych frameworków ML. Inspiracją był micrograd Andreja Karpathy'ego, ale z ambicją stworzenia biblioteki zdolnej do trenowania realnych modeli. Projekt wynika z filozofii tiny corp: "najmniejsze narzędzie, które działa". Hotz argumentuje, że współczesne biblioteki ML stały się zbyt złożone, co utrudnia innowacje i zrozumienie. Większość developerów nie potrzebuje wszystkich funkcji PyTorch'a, Tinygrad implementuje tylko niezbędne elementy.

## Podstawowe koncepcje i składnia tinygrad

Główną klasą jest `Tensor`, analogiczna do `torch.Tensor`. Na tensorach wykonujemy operacje element-po-elemencie (np. `x + y`), mnożenie macierzy (`x.matmul(y)`), stosujemy funkcje aktywacyjne (`.relu()`, `.sigmoid()`) oraz inne operacje (np. `reshape`, `permute`). Obliczenia są **leniwe** – wykonywane dopiero po poproszeniu o wynik, co pozwala tinygrad łączyć operacje w zoptymalizowane jądra obliczeniowe.

tinygrad wspiera **autograd**: wystarczy utworzyć tensor z `requires_grad=True`, wykonać operacje i wywołać `loss.backward()`, aby automatycznie obliczyć gradienty:


In [1]:
from tinygrad.tensor import Tensor
a = Tensor([1.0, 2.0, 3.0], requires_grad=True)
b = Tensor([4.0, 5.0, 6.0], requires_grad=True)
c = a * b + a
d = c.sum()
d.backward()
print("c =", c.numpy(), "∂d/∂a =", a.grad.numpy())

c = [ 5. 12. 21.] ∂d/∂a = [5. 6. 7.]


## Porównanie tinygrad z PyTorch

- **Interfejs (API)**: tinygrad celowo naśladuje PyTorch – składnia jest prawie identyczna, co ułatwia migrację:


In [2]:
# tinygrad
from tinygrad.tensor import Tensor
x = Tensor.eye(3, requires_grad=True)
y = Tensor([[2.0, 0, -2.0]], requires_grad=True)
z = y.matmul(x).sum()
z.backward()

# PyTorch
import torch
x = torch.eye(3, requires_grad=True)
y = torch.tensor([[2.0, 0, -2.0]], requires_grad=True)
z = y.matmul(x).sum()
z.backward()

- **Poziom abstrakcji**: PyTorch jest rozbudowanym frameworkiem wysokiego poziomu, podczas gdy tinygrad stawia na prostotę. Tinygrad **nie ma klasy `nn.Module`** – zamiast tego model to zwykła klasa Pythona, a propagację definiuje się przez `__call__`. Zachęca to do funkcjonalnego stylu programowania.

- **Wydajność**: PyTorch jest generalnie szybszy dzięki zoptymalizowanym bibliotekom. Tinygrad ma potencjalne zalety – może kompilować zestawy operacji do spersonalizowanych kerneli, a prostszy backend ułatwia optymalizację – ale obecnie nie dorównuje wydajnością.

- **Cel i zastosowania**: PyTorch jest skierowany na projekty produkcyjne, a tinygrad – przede wszystkim na **naukę i eksperymenty**. Jest także używany w zastosowaniach wbudowanych, gdzie lekkość implementacji ma znaczenie.

- **Implementacja nowych backendów**: Wiele firm tworzy obecnie własne akceleratory ML pod różne zastosowania (np. Google TPU, Apple Neural Engine, Groq, Graphcore). Aby PyTorch działał na tych akceleratorach, twórcy muszą zaimplementować około 200-300 operacji kernela. Dla TinyGrad jest to zaledwie 25-40 operacji.
  Przykład: TinyGrad nie ma dedykowanego kernela Dropout - implementuje go złożeniem kilku prostych operacji (losowa maska + mnożenie). Choć TinyGrad może być przez to nieco wolniejszy, pozwala firmom na znacznie szybszą implementację wsparcia dla akceleratora, a później stopniową optymalizację krytycznych ścieżek.
  Jest to analogiczne do architektury procesorów CISC (PyTorch - dużo wyspecjalizowanych instrukcji) vs RISC (TinyGrad - mała liczba uniwersalnych instrukcji), gdzie RISC upraszcza implementację procesora kosztem potencjalnie mniejszej wydajności niektórych operacji.

## Dlaczego wybrać tinygrad?

- **Łatwość zrozumienia**: tinygrad ma mały, czytelny kod źródłowy zawierający tylko niezbędne elementy.
- **Eksperymenty z akceleratorami**: zaprojektowany tak, by łatwo dodawać nowe backendy – każdy nowy akcelerator musi zaimplementować tylko kilkanaście podstawowych operacji.
- **Wsparcie GPU**: mimo prostoty, tinygrad może korzystać z GPU (OpenCL, CUDA, Triton) dla przyspieszenia obliczeń.
- **Ograniczenia**: nie jest przeznaczony do dużych, produkcyjnych projektów i nie ma tak bogatej biblioteki funkcji jak PyTorch.

## Architektura tinygrad

Tinygrad rozkłada skomplikowane obliczenia na **trzy podstawowe typy operacji**: elementarne, redukujące oraz przesunięcia danych. Wszystkie wyższe funkcje są zbudowane z kompozycji tych prostych bloków.

Obliczenia są wykonywane **leniwie**, co pozwala na fuzję wielu operacji w jeden program GPU/CPU. Dla przyspieszenia powtarzalnych fragmentów kodu, oferuje prosty JIT (dekorator `@TinyJit`).

Korzysta z prostych backendów: CPU realizuje operacje przez NumPy/Python, a GPU – przez OpenCL, CUDA, METAL itp. Dzięki temu wspiera wiele akceleratorów bez złożonej implementacji.

## Praktyczne zastosowanie – Klasyfikator MNIST

Poniżej kompletny przykład trenowania klasyfikatora MNIST w tinygrad:


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from time import time
import gc

gc.collect()
with torch.no_grad():
    torch.cuda.empty_cache()

class MNISTClassifier(nn.Module):
    def __init__(self):
        super(MNISTClassifier, self).__init__()
        self.l1 = nn.Conv2d(1, 32, kernel_size=(3, 3))
        self.l2 = nn.Conv2d(32, 64, kernel_size=(3, 3))
        self.l3 = nn.Linear(1600, 10)
    
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.l1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.l2(x)), (2, 2))
        x = x.flatten(1)
        x = F.dropout(x, 0.5, self.training)
        return self.l3(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

X_train = torch.stack([img for img, _ in train_dataset])
Y_train = torch.tensor([label for _, label in train_dataset])
X_test = torch.stack([img for img, _ in test_dataset])
Y_test = torch.tensor([label for _, label in test_dataset])

print(X_train.shape, X_train.dtype, Y_train.shape, Y_train.dtype)

model = MNISTClassifier().to(device)

optimizer = torch.optim.Adam(model.parameters())

start = time()
batch_size = 8
for step in range(2000):
    indices = torch.randint(0, len(X_train), (batch_size,))
    X, Y = X_train[indices].to(device), Y_train[indices].to(device)
    
    model.train()
    optimizer.zero_grad()
    outputs = model(X)
    loss = F.cross_entropy(outputs, Y)
    loss.backward()
    optimizer.step()
    
    if step % 100 == 0:
        model.eval()
        with torch.no_grad():
            outputs = model(X_test.to(device))
            predicted = outputs.argmax(dim=1)
            acc = (predicted == Y_test.to(device)).float().mean().item()
        print(f"step {step:4d}, loss {loss.item():.2f}, acc {acc*100:.2f}%")
print("total time", time()-start)

gc.collect()
with torch.no_grad():
    torch.cuda.empty_cache()

cuda
torch.Size([60000, 1, 28, 28]) torch.float32 torch.Size([60000]) torch.int64
step    0, loss 2.33, acc 15.90%
step  100, loss 0.29, acc 82.94%
step  200, loss 0.75, acc 89.55%
step  300, loss 0.53, acc 91.50%
step  400, loss 0.25, acc 92.70%
step  500, loss 0.42, acc 93.80%
step  600, loss 0.40, acc 94.98%
step  700, loss 0.04, acc 95.14%
step  800, loss 0.04, acc 95.55%
step  900, loss 0.08, acc 95.90%
step 1000, loss 0.01, acc 95.81%
step 1100, loss 0.09, acc 96.10%
step 1200, loss 0.01, acc 96.07%
step 1300, loss 0.07, acc 96.49%
step 1400, loss 1.00, acc 96.14%
step 1500, loss 0.02, acc 96.77%
step 1600, loss 0.01, acc 96.93%
step 1700, loss 0.06, acc 97.10%
step 1800, loss 0.02, acc 96.81%
step 1900, loss 0.02, acc 96.73%
total time 7.11711573600769


In [4]:
from tinygrad import Tensor, nn, TinyJit, Device
from tinygrad.nn.datasets import mnist
from time import time


class MNISTClassifier:
  def __init__(self):
    self.l1 = nn.Conv2d(1, 32, kernel_size=(3,3))
    self.l2 = nn.Conv2d(32, 64, kernel_size=(3,3))
    self.l3 = nn.Linear(1600, 10)

  def __call__(self, x:Tensor) -> Tensor:
    x = self.l1(x).relu().max_pool2d((2,2))
    x = self.l2(x).relu().max_pool2d((2,2))
    return self.l3(x.flatten(1).dropout(0.5))

print(Device.DEFAULT)

X_train, Y_train, X_test, Y_test = mnist()
print(X_train.shape, X_train.dtype, Y_train.shape, Y_train.dtype)

model = MNISTClassifier()
acc = (model(X_test).argmax(axis=1) == Y_test).mean()
# NOTE: tinygrad is lazy, and hasn't actually run anything by this point

optim = nn.optim.Adam(nn.state.get_parameters(model))
batch_size = 8

@TinyJit
def step():
  Tensor.training = True  # makes dropout work
  samples = Tensor.randint(batch_size, high=X_train.shape[0])
  X, Y = X_train[samples], Y_train[samples]
  optim.zero_grad()
  loss = model(X).sparse_categorical_crossentropy(Y).backward()
  optim.step()
  return loss


start = time()
for i in range(2000):
  loss = step()
  if i%100 == 0:
    Tensor.training = False
    acc = (model(X_test).argmax(axis=1) == Y_test).mean().item()
    print(f"acc {acc:.4f}")
print("total time", time()-start)

CUDA
(60000, 1, 28, 28) dtypes.uchar (60000,) dtypes.uchar
acc 0.1283
acc 0.8502
acc 0.8838
acc 0.9171
acc 0.8810
acc 0.9078
acc 0.9333
acc 0.9148
acc 0.9319
acc 0.9385
acc 0.9258
acc 0.9365
acc 0.9333
acc 0.9456
acc 0.9485
acc 0.9486
acc 0.9463
acc 0.9426
acc 0.9580
acc 0.9446
total time 31.083072900772095


## Zastosowania w projektach rzeczywistych

Przykłady wykorzystania tinygrad w praktyce:

- OpenPilot (komponent samojezdnego samochodu)
- Mobilne przetwarzanie obrazów
- Projekty edukacyjne wymagające zrozumienia całego stosu ML

## Najważniejsze różnice z kulturą PyTorch i TensorFlow

PyTorch/TensorFlow promują używanie gotowych abstrakcji wysokiego poziomu (modele, warstwy). tinygrad zachęca do budowania wszystkiego od podstaw - rozumienie, a nie tylko użycie. Różnica filozoficzna to również podejście do debugowania - w tinygrad widać wszystkie operacje, podczas gdy w większych bibliotekach wiele operacji dzieje się "pod maską".

W tinygrad możemy użyć `with Context(DEBUG=2)`, aby zobaczyć wszystkie kernele, które są wykonywane. DEBUG=4, pokazuje cały kod, w PyTorch jest to ciężkie.

## Przyszłość tinygrad

Tinycorp regularnie rozwija projekt, dodając:

- Wsparcie dla nowych architektur (Apple M1/M2)
- Optymalizacje kompilatora (JIT)
- Wsparcie dla nowych modeli (np. LLM, stable diffusion)

## Zasoby do dalszego zgłębienia

- Oficjalne repozytorium: https://github.com/tinygrad/tinygrad
- Kanał YouTube George'a Hotza: omówienia implementacji https://www.youtube.com/@geohotarchive/videos
- Dokumentacja: https://docs.tinygrad.org/

## Podsumowanie

tinygrad to **bardzo lekki** framework, którego głównym celem jest edukacja i zrozumienie działania głębokiego uczenia. Oferuje uproszczony interfejs podobny do PyTorch, ale rezygnuje z wielu abstrakcji. Najważniejsze zalety: prostota implementacji, czytelność kodu i łatwość rozbudowy. Stanowi interesującą alternatywę do nauki mechanizmów uczenia, pokazując „od podszewki" jak działa propagacja wsteczna i przetwarzanie tensorów.
